In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth, GeoJson, GeoJsonTooltip
import datetime
import branca
import locale
import os
import shapely
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString
from matplotlib import colors, colormaps
from functools import partial
from ipywidgets import widgets, Dropdown, RadioButtons, Output, VBox, HBox
from IPython.display import display, clear_output

from widgets import create_slider, create_buttons, create_radio_buttons, create_dropdowns, create_labels

<h1>Carregando shapefiles (.shp) referentes às arestas, zonas e distritos</h1>

In [2]:
hexs = gpd.read_file('hexs/base_hexs.shp')

edges = gpd.read_file('edges/base_edges.shp')
edges.crs = 'epsg:4326'

zonas_od = gpd.read_file('OD-2017/Mapas-OD2017/Shape-OD2017/Zonas_2017_region.shp')
zonas_sp = zonas_od[zonas_od['NumeroMuni'] == 36]

municipios_sp = gpd.read_file('SP_Municipios_2022/SP_Municipios_2022.shp')
sp = municipios_sp[municipios_sp['NM_MUN'] == 'São Paulo']

# distritos = gpd.read_file('GeoSampa/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.shp')
# subprefeituras = gpd.read_file('GeoSampa/SIRGAS_SHP_subprefeitura/SIRGAS_SHP_subprefeitura_polygon.shp')
distritos = gpd.read_file('edited shapefiles/edited_distritos.shp')
subprefeituras = gpd.read_file('edited shapefiles/edited_subprefeituras.shp')
distritos.set_crs(epsg=31983, inplace=True)
subprefeituras.set_crs(epsg=31983, inplace=True)

if edges.crs != zonas_sp.crs:
    zonas_sp = zonas_sp.to_crs(edges.crs)
if edges.crs != distritos.crs:
    distritos = distritos.to_crs(edges.crs)
if edges.crs != subprefeituras.crs:
    subprefeituras = subprefeituras.to_crs(edges.crs)

combined_df_edges = pd.DataFrame()
combined_df_edges_g = pd.DataFrame()
combined_df_edges_c = pd.DataFrame()
combined_df_edges_quartile = pd.DataFrame()
combined_df_edges_vel = pd.DataFrame()
combined_df_hex_o = pd.DataFrame()
combined_df_hex_d = pd.DataFrame()

In [3]:
ciclovias = gpd.read_file('Estrutura cicloviária SP/Ciclovias.shp')
ciclovias = ciclovias.drop(columns='inauguracao')
ciclovias.crs = 'epsg:4326'

ciclorrotas = gpd.read_file('Estrutura cicloviária SP/Ciclorrotas.shp')
ciclorrotas = ciclorrotas.drop(columns='inauguracao')
ciclorrotas.crs = 'epsg:4326'

conexoes = gpd.read_file('Estrutura cicloviária SP/CicloviasConexoes.shp')
conexoes.crs = 'epsg:4326'

if ciclovias.crs.is_geographic:
    ciclovias = ciclovias.to_crs(epsg=32723) 
if ciclorrotas.crs.is_geographic:
    ciclorrotas = ciclorrotas.to_crs(epsg=32723) 
if conexoes.crs.is_geographic:
    conexoes = conexoes.to_crs(epsg=32723) 

<h2>Modelos mapas por limite administrativo</h2>

In [4]:
# Mapa padrão - distritos
m_sp = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    sp,
    name='São Paulo',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'black',
        'fillOpacity': 0.1
    }
).add_to(m_sp)

# Mapa padrão - distritos
m_distritos = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    distritos,
    name='Distritos',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'black',
        'fillOpacity': 0.1
    }
).add_to(m_distritos)

# Mapa padrão - zonas
m_zonas = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    zonas_sp,
    name='Zonas',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'black',
        'fillOpacity': 0.1
    }
).add_to(m_zonas)

# Mapa padrão - subprefeituras
m_subprefeituras = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    subprefeituras,
    name='Subprefeituras',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'black',
        'fillOpacity': 0.1
    }
).add_to(m_subprefeituras)

<h2>Zonas OD / Distritos / Subprefeituras</h2>

In [30]:
# Criação de Outputs
map_output = Output()
table_output = Output()
dropdown_output = Output()
debug_output = Output()
notification_output = Output()
ui_output = Output()

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Criação de Radio Buttons
granularidade = create_radio_buttons(['Geral', 'Zonas', 'Distritos', 'Subprefeituras'], 'Granularidade:')
filtro = create_radio_buttons(['Nenhum', 'Idade', 'Gênero'], 'Filtrar por:')
idade = create_radio_buttons(['Qualquer', '18-34', '35-54', '55-64', '65+'], 'Idade:')
genero = create_radio_buttons(['Qualquer', 'Feminino', 'Masculino', 'Não especificado'], 'Gênero:')

state = {'granularidade': granularidade.value, 'filtro': filtro.value, 'idade': idade.value, 'genero': genero.value}

# Criação de Dropdowns
sorted_zonas = zonas_sp.sort_values(by='NomeZona')
zone_dropdown = create_dropdowns([('Selecione uma zona', None)] + [(f"{row['NomeZona']}", row['NumeroZona']) for index, row in sorted_zonas.iterrows()], 'Zona:')
distritos_dropdown = create_dropdowns([('Selecione um distrito', None)] + [(f"{row['ds_nome']}", row['ds_codigo']) for index, row in distritos.iterrows()], 'Distritos:')
subprefeituras_dropdown = create_dropdowns([('Selecione uma subprefeitura', None)] + [(f"{row['sp_nome']}", row['sp_codigo']) for index, row in subprefeituras.iterrows()], 'Subprefeituras')

# Criação de Widgets para o filtro temporal
layout = widgets.Layout(width='100px')
start_month = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
end_month = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
start_year = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
end_year = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
label = create_labels('Dados disponíveis: 01/2019 à 08/2024', layout=widgets.Layout(width='250px'))
label1 = create_labels('Início do período', layout=widgets.Layout(width='100px'))
label2 = create_labels('Fim do período', layout=widgets.Layout(width='100px'))
button = create_buttons('Selecionar período')

#Filtra o período ao pressionar o botão
def on_button_click(b):
    stt_y = start_year.value 
    end_y = end_year.value 
    stt_m = start_month.value
    end_m = end_month.value

    with notification_output:
        notification_output.clear_output(wait=True)
        
    with ui_output:
        ui_output.clear_output(wait=True)

    start_date = f"{stt_y} {stt_m}"
    end_date = f"{end_y} {end_m}"

    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)

    limit_2024 = pd.to_datetime('2024-08')
    if end_date_dt > limit_2024:
        with notification_output:
            print("Erro: Os dados estão disponíveis apenas até 08/2024.")
        return

    if end_date_dt < start_date_dt:
        with notification_output:
            print("Erro: Período inválido. Verifique início e fim do período.")
        return

    matching_files = []
    
    for year in range(int(stt_y), int(end_y) + 1):
        for month in months:
            filename = f"{year} {month}.csv"
            file_path = os.path.join('csvs-editados', filename)
            if year > int(stt_y) or (year == int(stt_y) and month >= stt_m):
                if year < int(end_y) or (year == int(end_y) and month <= end_m):
                    if os.path.isfile(file_path):
                        matching_files.append(file_path)

    if matching_files:
        try:
            with notification_output:
                print("Obtendo dados...")
            
            df_list = [pd.read_csv(file) for file in matching_files] 
            concatenated_df = pd.concat(df_list, ignore_index=True)

            global combined_df_edges
            combined_df_edges = concatenated_df 
            
            with notification_output:
                print(f"Dados de {stt_m}/{stt_y} a {end_m}/{end_y} obtidos.")

            with ui_output:
                display(HBox([granularidade, filtro, idade, genero]), dropdown_output, VBox([map_output, table_output, debug_output]))
            
        except Exception as e:
            with notification_output:
                print(f"Erro na obtenção de dados: {e}")
       
    else:
        with notification_output:
            print("Nenhum arquivo encontrado.")

# Define a cor de cada edge
def get_color(grouped, uid, min_val, max_val, col, count):
    if uid not in grouped[col].values:
        norm = colors.Normalize(vmin=min_val, vmax=max_val)
        color_value = norm(min_val)
        return colors.to_hex(cmap(color_value))
    
    total_trips = grouped[grouped[col] == uid][count].values[0]
    norm = colors.Normalize(vmin=min_val, vmax=max_val)
    
    color_value = norm(total_trips)
    return colors.to_hex(cmap(color_value))

# Normalização de valores das escalas
def get_normalization_values(grouped_edges):
    min_val = grouped_edges['total_viagens'].min()
    max_val = grouped_edges['total_viagens'].max()
    return min_val, max_val

# Mudança no mapa por granularidade
def update_map(granularidade):
    with map_output:
        map_output.clear_output(wait=True)
        
        if granularidade == "Geral":
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
            zone_dropdown.layout.display = 'none'
            distritos_dropdown.layout.display = 'none'
            subprefeituras_dropdown.layout.display = 'none'
            display(m_sp)
            
        elif granularidade == 'Zonas':
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
                zone_dropdown.layout.display = 'block'
                display(zone_dropdown)
            zone_dropdown.value = None
            display(m_zonas)

        elif granularidade == 'Distritos':
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
                distritos_dropdown.layout.display = 'block'
                display(distritos_dropdown)
            distritos_dropdown.value = None
            display(m_distritos)
            
        elif granularidade == 'Subprefeituras':
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
                subprefeituras_dropdown.layout.display = 'block'
                display(subprefeituras_dropdown)
            subprefeituras_dropdown.value = None
            display(m_subprefeituras)

def compute_trip_counts(grouped_df):
    trips = grouped_df[['total_viagens', 'total_viagens_lazer', 'total_viagens_commute', 'total_pessoas',
                        'total_pessoas_18_34', 'total_pessoas_35_54', 'total_pessoas_55_64', 'total_pessoas_o65']].sum().reset_index()
    
    return trips


# Obtenção de dados do período selecionado
def get_data(df, value):
    if value == "Zonas":
        edges_in_zones = gpd.sjoin(edges, zonas_sp, how="inner", predicate="intersects")
        df_merged_zones = pd.merge(df, edges_in_zones[['edgeUID', 'NumeroZona', 'NomeZona']], left_on='edge_uid', right_on='edgeUID', how='inner')
        grouped_edges_in_zones = df_merged_zones.groupby(['edgeUID', 'NumeroZona', 'NomeZona']).agg(
            total_viagens=('total_viagens', 'sum'),
            total_pessoas=('total_pessoas', 'sum'),
            total_pessoas_18_34=('total_pessoas_18_34', 'sum'),
            total_pessoas_35_54=('total_pessoas_35_54', 'sum'),
            total_pessoas_55_64=('total_pessoas_55_64', 'sum'),
            total_pessoas_o65=('total_pessoas_o65', 'sum'),
            total_pessoas_masculino=('total_pessoas_masculino', 'sum'),
            total_pessoas_feminino=('total_pessoas_feminino', 'sum'),
            total_pessoas_nao_especificado=('total_pessoas_nao_especificado', 'sum'),
        ).reset_index()
        grouped_zones = df_merged_zones.groupby(['NumeroZona', 'NomeZona'])
        trips_per_zone = compute_trip_counts(grouped_zones)
        
        return grouped_edges_in_zones, trips_per_zone

    elif value == "Distritos":
        edges_in_districts = gpd.sjoin(edges, distritos, how="inner", predicate="intersects")
        df_merged_districts = pd.merge(df, edges_in_districts[['edgeUID', 'ds_nome', 'ds_codigo']], left_on='edge_uid', right_on='edgeUID', how='inner')
        grouped_edges_in_districts = df_merged_districts.groupby(['edgeUID', 'ds_codigo', 'ds_nome']).agg(
            total_viagens=('total_viagens', 'sum'),
            total_pessoas=('total_pessoas', 'sum'),
            total_pessoas_18_34=('total_pessoas_18_34', 'sum'),
            total_pessoas_35_54=('total_pessoas_35_54', 'sum'),
            total_pessoas_55_64=('total_pessoas_55_64', 'sum'),
            total_pessoas_o65=('total_pessoas_o65', 'sum'),
            total_pessoas_masculino=('total_pessoas_masculino', 'sum'),
            total_pessoas_feminino=('total_pessoas_feminino', 'sum'),
            total_pessoas_nao_especificado=('total_pessoas_nao_especificado', 'sum'),
        ).reset_index()
        grouped_districts = df_merged_districts.groupby(['ds_nome'])
        trips_per_district = compute_trip_counts(grouped_districts)

        return grouped_edges_in_districts, trips_per_district

    elif value == "Subprefeituras":
        edges_in_subprefeituras = gpd.sjoin(edges, subprefeituras, how="inner", predicate="intersects")
        df_merged_subprefeituras = pd.merge(df, edges_in_subprefeituras[['edgeUID', 'sp_nome', 'sp_codigo']], left_on='edge_uid', right_on='edgeUID', how='inner')
        grouped_edges_in_subprefeituras = df_merged_subprefeituras.groupby(['edgeUID', 'sp_codigo', 'sp_nome']).agg(
            total_viagens=('total_viagens', 'sum'),
            total_pessoas=('total_pessoas', 'sum'),
            total_pessoas_18_34=('total_pessoas_18_34', 'sum'),
            total_pessoas_35_54=('total_pessoas_35_54', 'sum'),
            total_pessoas_55_64=('total_pessoas_55_64', 'sum'),
            total_pessoas_o65=('total_pessoas_o65', 'sum'),
            total_pessoas_masculino=('total_pessoas_masculino', 'sum'),
            total_pessoas_feminino=('total_pessoas_feminino', 'sum'),
            total_pessoas_nao_especificado=('total_pessoas_nao_especificado', 'sum'),
        ).reset_index()
        grouped_subprefeituras = df_merged_subprefeituras.groupby(['sp_nome'])
        trips_per_subprefeitura = compute_trip_counts(grouped_subprefeituras)

        return grouped_edges_in_subprefeituras, trips_per_subprefeitura

# Mudança de zona/distrito/subprefeitura a partir da seleção do usuário
def update_loc(gran, age = None, gender = None):
    
    global combined_df_edges
    if combined_df_edges.empty:
        with notification_output:
            notification_output.clear_output()
            print('ERRO: nenhum período selecionado. Por favor selecione um período.')
    
    loc_id = gran
        
    if loc_id is None:
        return
        
    selected_loc_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
    
    if granularidade.value == "Zonas":
        full_map = zonas_sp
        grouped_edges, trips_per_loc = get_data(combined_df_edges, "Zonas")
        sufix = 'da Zona'
        selected_loc = zonas_sp[zonas_sp["NumeroZona"] == loc_id]
        name = selected_loc['NomeZona'].iloc[0]
        table_data = trips_per_loc[trips_per_loc['NumeroZona'] == loc_id].reset_index(drop=True)
        table_data = table_data.rename(columns={'NomeZona': 'Nome da Zona', 
                                                'total_pessoas': 'Total de pessoas'})
        
    elif granularidade.value == "Distritos":
        full_map = distritos
        grouped_edges, trips_per_loc = get_data(combined_df_edges, "Distritos")
        sufix = 'do Distrito'
        selected_loc = distritos[distritos["ds_codigo"] == loc_id]
        name = selected_loc['ds_nome'].iloc[0]
        table_data = trips_per_loc[trips_per_loc['ds_nome'] == name].reset_index(drop=True)
        table_data = table_data.rename(columns={'ds_nome': 'Nome do distrito', 
                                                'total_pessoas': 'Total de pessoas'})
        
    elif granularidade.value == "Subprefeituras":
        full_map = subprefeituras
        grouped_edges, trips_per_loc = get_data(combined_df_edges, "Subprefeituras")
        sufix = 'da Subprefeitura'
        selected_loc = subprefeituras[subprefeituras["sp_codigo"] == loc_id]
        name = selected_loc['sp_nome'].iloc[0]
        table_data = trips_per_loc[trips_per_loc['sp_nome'] == name].reset_index(drop=True)
        table_data = table_data.rename(columns={'sp_nome': 'Nome da subprefeitura', 
                                                'total_pessoas': 'Total de pessoas'})

    if selected_loc.empty:
        with debug_output:
            print("No matching data found for the selected loc_id.")
        return

    filtered_edges = gpd.overlay(edges, selected_loc, how="intersection")
    selected_edges = grouped_edges[grouped_edges['edgeUID'].isin(filtered_edges['edgeUID'])]

    #if age is not None:
    if age != "Qualquer":
        if age == '18-34':
            col = 'total_pessoas_18_34'
        elif age == '35-54':
            col = 'total_pessoas_35_54'
        elif age == '55-64':
            col = 'total_pessoas_55_64'
        else:
            col = 'total_pessoas_o65'
        min_val, max_val = selected_edges[col].min(), selected_edges[col].max()
        table_data = table_data[[col for col in table_data.columns if col in ['Nome da Zona', 'Nome do distrito', 'Nome da subprefeitura', 'Total de pessoas', age]]]
    
        # if age == "Qualquer":
        #     col = 'total_pessoas'
        #     min_val, max_val = selected_edges['total_pessoas'].min(), selected_edges['total_pessoas'].max()
        #     #table_data = table_data[[col for col in table_data.columns if col in ['Nome da Zona', 'Nome do distrito', 'Nome da subprefeitura', 'Total de viagens']]]
    
    elif gender != "Qualquer":
        with notification_output:
            print('here')
        if gender == "Masculino":
            col = 'total_pessoas_masculino'
        elif gender == "Feminino":
            col = 'total_pessoas_feminino'
        else:
            col = 'total_pessoas_nao_especificado'

        min_val, max_val = selected_edges[col].min(), selected_edges[col].max()
        table_data = table_data[[col for col in table_data.columns if col in ['Nome da Zona', 'Nome do distrito', 'Nome da subprefeitura', 'Total de pessoas', gender]]]
            
    if gender == "Qualquer" and age == "Qualquer":
        col = 'total_pessoas'
        min_val, max_val = selected_edges['total_pessoas'].min(), selected_edges['total_pessoas'].max()

    filtered_edges = filtered_edges.merge(
        selected_edges[['edgeUID', col]],  # Seleciona apenas os campos necessários
        on='edgeUID',                                  # Campo comum para realizar a junção
        how='left'                                     # Mantém todos os registros de filtered_edges
    )
    
    # realça a zona/distrito/subprefeitura escolhida
    folium.GeoJson(
        selected_loc,
        style_function=lambda feature: {
            'weight': 0,
            'color': 'black',
            'fillOpacity': 0.2
        },
        name = 'Limite administrativo selecionado'
    ).add_to(selected_loc_map)
    
    folium.GeoJson(
            filtered_edges,
            style_function=lambda feature: {
                'weight': 2,
                'color': get_color(selected_edges, feature['properties']['edgeUID'], min_val, max_val, 'edgeUID', col)
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['edgeUID', col],
                aliases=['ID do Edge:', 'Total de pessoas:'],
                sticky=True,
                localize=True,
                labels=True,
                formatters={
                    'edgeUID': 'function() { return this.edgeUID; }',
                }
            ),
            name='Arestas dentro do limite administrativo'
        ).add_to(selected_loc_map)

    c_ciclovias = ciclovias[ciclovias['programa'].str.contains("CICLOVIA", case=False, na=False)]
    c_ciclofaixas = ciclovias[ciclovias['programa'].str.contains("CICLOFAIXA", case=False, na=False)]

    folium.GeoJson(
        c_ciclovias,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'green'
            },
        name="Ciclovias",
        show=False 
    ).add_to(selected_loc_map)

    folium.GeoJson(
        c_ciclofaixas,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'blue'
            },
        name="Ciclofaixas",
        show=False 
    ).add_to(selected_loc_map)

    folium.GeoJson(
        ciclorrotas,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'purple'
            },
        name="Ciclorrotas",
        show=False 
    ).add_to(selected_loc_map)

    folium.GeoJson(
        conexoes,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'pink'
            },
        name="Conexões",
        show=False 
    ).add_to(selected_loc_map)
    
    folium.LayerControl().add_to(selected_loc_map)

    # Limites geográficos (bounds) zona/distrito/subprefeitura para focar visualização (zoom)
    bounds = selected_loc.geometry.total_bounds
    sw = [bounds[1], bounds[0]]
    ne = [bounds[3], bounds[2]]
    selected_loc_map.fit_bounds([sw, ne])

    branca_cmap = branca.colormap.LinearColormap(
        colors=[colors.to_hex(cmap(i)) for i in range(cmap.N)], 
        vmin=0, vmax=max_val
    )

    branca_cmap.caption = 'Total de pessoas'
    branca_cmap.add_to(selected_loc_map)

    with map_output:
        map_output.clear_output(wait=True)
        display(selected_loc_map)
        
    styled_table = table_data.style \
        .hide(axis='index') \
        .set_table_styles({
            f'Nome {sufix}': [{'selector': 'td, th', 'props': 'width: 200px; text-align: center;'}],
            'Total de pessoas': [{'selector': 'td, th', 'props': 'width: 100px; text-align: center;'}]
        }) \
        .set_properties(**{'text-align': 'center', 'padding': '10px'}) \
        .format(thousands=".")

    with table_output:
        table_output.clear_output(wait=True)
        display(styled_table)

# ações com mudança dos Radio Butttons
def on_value_change(change):
    if change['owner'] is granularidade:
        state['granularidade'] = change['new']
        update_map(state['granularidade'])
    if change['owner'] is filtro:
        state['filtro'] = change['new']
        if filtro.value == 'Nenhum':
            loc_id = None
            idade.value = "Qualquer"
            genero.value = "Qualquer"
            if granularidade.value == 'Zonas':
                loc_id = zone_dropdown.value
            elif granularidade.value == 'Distritos':
                loc_id = distritos_dropdown.value
            elif granularidade.value == 'Subprefeituras':
                loc_id = subprefeituras_dropdown.value
            idade.layout.display = 'none'
            genero.layout.display = 'none'
            update_loc(loc_id, state['idade'], state['genero'])
        elif filtro.value == 'Idade':
            loc_id = None
            idade.value = "Qualquer"
            genero.value = "Qualquer"
            if granularidade.value == 'Zonas':
                loc_id = zone_dropdown.value
            elif granularidade.value == 'Distritos':
                loc_id = distritos_dropdown.value
            elif granularidade.value == 'Subprefeituras':
                loc_id = subprefeituras_dropdown.value
            idade.layout.display = ''
            genero.layout.display = 'none'
            #update_loc(loc_id, state['idade'], state['genero'])
        elif filtro.value == 'Gênero':
            loc_id = None
            idade.value = "Qualquer"
            genero.value = "Qualquer"
            if granularidade.value == 'Zonas':
                loc_id = zone_dropdown.value
            elif granularidade.value == 'Distritos':
                loc_id = distritos_dropdown.value
            elif granularidade.value == 'Subprefeituras':
                loc_id = subprefeituras_dropdown.value
            idade.layout.display = 'none'
            genero.layout.display = ''
            #update_loc(loc_id, state['idade'], state['genero'])
    elif change['owner'] is idade:
        state['idade'] = change['new']
        loc_id = None
        if granularidade.value == 'Zonas':
            loc_id = zone_dropdown.value
        elif granularidade.value == 'Distritos':
            loc_id = distritos_dropdown.value
        elif granularidade.value == 'Subprefeituras':
            loc_id = subprefeituras_dropdown.value
        update_loc(loc_id, state['idade'], state['genero'])
    elif change['owner'] is genero:
        state['genero'] = change['new']
        loc_id = None
        if granularidade.value == 'Zonas':
            loc_id = zone_dropdown.value
        elif granularidade.value == 'Distritos':
            loc_id = distritos_dropdown.value
        elif granularidade.value == 'Subprefeituras':
            loc_id = subprefeituras_dropdown.value
        update_loc(loc_id, state['idade'], state['genero'])
        
# Radio Buttons Observers 
granularidade.observe(on_value_change, names='value')
filtro.observe(on_value_change, names='value')
idade.observe(on_value_change, names='value')
genero.observe(on_value_change, names='value')

# Dropdown Observers
zone_dropdown.observe(lambda change: update_loc(change['new'], state['idade'], state['genero']), names='value')
distritos_dropdown.observe(lambda change: update_loc(change['new'], state['idade'], state['genero']), names='value')
subprefeituras_dropdown.observe(lambda change: update_loc(change['new'], state['idade'], state['genero']), names='value')

button.on_click(on_button_click)

idade.layout.display = 'none'
genero.layout.display = 'none'

update_map(state['granularidade'])
    
cmap = colormaps.get_cmap('YlOrBr')

#display(HBox([label1, start_month, start_year]), HBox([label2, end_month, end_year]), button, notification_output, HBox([granularidade, idade, genero]), dropdown_output, VBox([map_output, table_output, debug_output]))
display(label, HBox([label1, start_month, start_year]), HBox([label2, end_month, end_year]), button, notification_output, ui_output, debug_output)

Label(value='Dados disponíveis: 01/2019 à 08/2024', layout=Layout(width='250px'))

Button(description='Selecionar período', style=ButtonStyle())

Output()

Output()

Output()

<h2>Origens e Destinos por limite administrativo</h2>

In [6]:
od_map_output = Output()
od_dropdown_output = Output()
debug2 = Output()
notification_output_od = Output()
ui_output_od = Output()

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

granularidade_od = create_radio_buttons(['Geral', 'Zonas', 'Distritos', 'Subprefeituras'], 'Granularidade:')
tipo = create_radio_buttons(['Origens', 'Destinos'], 'Tipo:')
periodo = create_radio_buttons(['Qualquer', '05:00-10:00', '10:00-15:00', '15:00-20:00', '20:00-05:00'], 'Período do dia:')
zone_dropdown_od = create_dropdowns([('Selecione uma zona', None)] + [(f"{row['NomeZona']}", row['NumeroZona']) for index, row in sorted_zonas.iterrows()], 'Zona:')
distritos_dropdown_od = create_dropdowns([('Selecione um distrito', None)] + [(f"{row['ds_nome']}", row['ds_codigo']) for index, row in distritos.iterrows()], 'Distritos:')
subprefeituras_dropdown_od = create_dropdowns([('Selecione uma subprefeitura', None)] + [(f"{row['sp_nome']}", row['sp_codigo']) for index, row in subprefeituras.iterrows()], 'Subprefeituras')

state.update({'granularidade_od': granularidade_od.value, 'tipo': tipo.value, 'periodo': periodo.value})

# Criação de Widgets para o filtro temporal
layout = widgets.Layout(width='100px')
start_month_od = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
end_month_od = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
start_year_od = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
end_year_od = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
label_od = create_labels('Dados disponíveis: 01/2019 à 08/2024', layout=widgets.Layout(width='250px'))
label1_od = create_labels('Início do período', layout=widgets.Layout(width='100px'))
label2_od = create_labels('Fim do período', layout=widgets.Layout(width='100px'))
button_od = create_buttons('Selecionar período')

#display(HBox([granularidade_od, tipo, periodo]), od_dropdown_output, od_map_output, debug2)

def on_button_click_od(b):
    with notification_output_od:
        notification_output_od.clear_output()

    with ui_output_od:
        ui_output_od.clear_output()
        
    start_date = f"{start_year_od.value} {start_month_od.value}"
    end_date = f"{end_year_od.value} {end_month_od.value}"

    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)

    limit_2024 = pd.to_datetime('2024-08')

    if end_date_dt > limit_2024:
        with notification_output_od:
            print("Erro: Os dados estão disponíveis apenas até 08/2024.")
        return

    if end_date_dt < start_date_dt:
        with notification_output_od:
            print("Erro: Período inválido. Verifique início e fim do período.")
        return

    matching_files_o = []
    matching_files_d = []
    for year in range(int(start_year_od.value), int(end_year_od.value) + 1):
        for month in months:
            filename_o = f"{year} {month}_o.csv"
            filename_d = f"{year} {month}_d.csv"
            file_path_o = os.path.join(f'csvs-hex\\{year} {month}', filename_o)
            file_path_d = os.path.join(f'csvs-hex\\{year} {month}', filename_d)
            if year > int(start_year_od.value) or (year == int(start_year_od.value) and month >= start_month_od.value):
                if year < int(end_year_od.value) or (year == int(end_year_od.value) and month <= end_month_od.value):
                    if os.path.isfile(file_path_o):
                        matching_files_o.append(file_path_o)
                    if os.path.isfile(file_path_d):
                        matching_files_d.append(file_path_d)
    
    if matching_files_o and matching_files_d:
        try:
            with notification_output_od:
                print("Obtendo dados...")
            
            df_list_o = [pd.read_csv(file) for file in matching_files_o] 
            df_list_d = [pd.read_csv(file) for file in matching_files_d] 
            concatenated_df_o = pd.concat(df_list_o, ignore_index=True)
            concatenated_df_d = pd.concat(df_list_d, ignore_index=True)

            global combined_df_hex_o
            global combined_df_hex_d
            combined_df_hex_o = concatenated_df_o 
            combined_df_hex_d = concatenated_df_d 
            
            #output_file = "concatenated_data.csv" 
            #concatenated_df.to_csv(output_file, index=False) 
            with notification_output_od:
                print(f"Dados de {start_month_od.value}/{start_year_od.value} a {end_month_od.value}/{end_year_od.value} obtidos.")

            with ui_output_od:
                display(HBox([granularidade_od, tipo, periodo]), od_dropdown_output, od_map_output)
            
        except Exception as e:
            with notification_output_od:
                print(f"Erro na obtenção de dados: {e}")
       
    else:
        with notification_output_od:
            print("Nenhum arquivo encontrado.")

def update_map_od(granularidade_od):
    with od_map_output:
        od_map_output.clear_output(wait=True)
        
        if granularidade_od == "Geral":
            with od_dropdown_output:
                od_dropdown_output.clear_output(wait=True)
            zone_dropdown_od.layout.display = 'none'
            distritos_dropdown_od.layout.display = 'none'
            subprefeituras_dropdown_od.layout.display = 'none'
            display(m_sp)
            
        elif granularidade_od == 'Zonas':
            with od_dropdown_output:
                od_dropdown_output.clear_output(wait=True)
                zone_dropdown_od.layout.display = 'block'
                display(zone_dropdown_od)
            zone_dropdown_od.value = None
            display(m_zonas)

        elif granularidade_od == 'Distritos':
            with od_dropdown_output:
                od_dropdown_output.clear_output(wait=True)
                distritos_dropdown_od.layout.display = 'block'
                display(distritos_dropdown_od)
            distritos_dropdown_od.value = None
            display(m_distritos)
            
        elif granularidade_od == 'Subprefeituras':
            with od_dropdown_output:
                od_dropdown_output.clear_output(wait=True)
                subprefeituras_dropdown_od.layout.display = 'block'
                display(subprefeituras_dropdown_od)
            subprefeituras_dropdown_od.value = None
            display(m_subprefeituras)

def get_data_od(df, gran, cod, nome):
    desired_hexs = gpd.sjoin(hexs, gran, how="inner", predicate="intersects")
    df_merged = pd.merge(df, desired_hexs[['hex_id', cod, nome]], left_on='hex_id', right_on='hex_id', how='inner')
    grouped_hexs = df_merged.groupby(['hex_id', cod, nome]).agg(
        count=('count', 'sum'),
        morning_count=('morning_count', 'sum'),
        midday_count=('midday_count', 'sum'),
        evening_count=('evening_count', 'sum'),
        overnight_count=('overnight_count', 'sum'),
    ).reset_index()

    return grouped_hexs

def update_hexs(gran, tipo, periodo):
    global combined_df_hex_o
    global combined_df_hex_d
    if tipo == "Origens":
        data = combined_df_hex_o
    else:
        data = combined_df_hex_d
    
    if data.empty:
        with notification_output_od:
            notification_output_od.clear_output()
            print('ERRO: nenhum período selecionado. Por favor selecione um período.')
    
    loc_id = gran
    if loc_id is None:
        return

    hexs_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
    
    if granularidade_od.value == "Zonas":
        grouped_hexs = get_data_od(data, zonas_sp, 'NumeroZona', 'NomeZona')
        selected_loc = zonas_sp[zonas_sp["NumeroZona"] == loc_id]
        
    elif granularidade_od.value == "Distritos":
        grouped_hexs = get_data_od(data, distritos, 'ds_codigo', 'ds_nome')
        selected_loc = distritos[distritos["ds_codigo"] == loc_id]
        
    elif granularidade_od.value == "Subprefeituras":
        grouped_hexs = get_data_od(data, subprefeituras, 'sp_codigo', 'sp_nome')
        selected_loc = subprefeituras[subprefeituras["sp_codigo"] == loc_id]

    filtered_hexs = gpd.overlay(hexs, selected_loc, how="intersection")
    selected_hexs = grouped_hexs[grouped_hexs['hex_id'].isin(filtered_hexs['hex_id'])]

    if periodo is not None and periodo != "Qualquer":
        if periodo == '05:00-10:00':
            periodo_col = 'morning_count'
            
        elif periodo == '10:00-15:00':
            periodo_col = 'midday_count'
            
        elif periodo == '15:00-20:00':
            periodo_col = 'evening_count'
            
        else:
            periodo_col = 'overnight_count'
            
        min_val, max_val = selected_hexs[periodo_col].min(), selected_hexs[periodo_col].max()
        #table_data = table_data[[col for col in table_data.columns if col in ['Nome da Zona', 'Nome do distrito', 'Nome da subprefeitura', 'Total de viagens', periodo_col]]]

    if periodo == 'Qualquer':
        periodo_col = 'count'
        min_val, max_val = selected_hexs['count'].min(), selected_hexs['count'].max()

    filtered_hexs = filtered_hexs.merge(
        selected_hexs[['hex_id', periodo_col]],  
        on='hex_id',                
        how='left'                      
    )

    folium.GeoJson(
            filtered_hexs,
            style_function=lambda feature: {
                'weight': 0.5,
                'color': 'black',
                'fillColor': get_color(selected_hexs, feature['properties']['hex_id'], min_val, max_val, 'hex_id', periodo_col),
                'fillOpacity': 0.7
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['hex_id', periodo_col],
                aliases=['ID do Hexágono:', 'Total de atividades:'],
                sticky=True,
                localize=True,
                labels=True
            )
        ).add_to(hexs_map)

    # folium.GeoJson(
    #     filtered_hexs,
    #     style_function=lambda feature: {
    #         'weight': 0.5,
    #         'color': 'black',
    #         'fillColor': get_color(selected_hexs, feature['properties']['hex_id'], min_val, max_val, 'hex_id', periodo_col),
    #         'fillOpacity': 0.7
    #     }
    # ).add_to(hexs_map)

    # Limites geográficos (bounds) zona/distrito/subprefeitura para focar visualização (zoom)
    bounds = selected_loc.geometry.total_bounds
    sw = [bounds[1], bounds[0]]
    ne = [bounds[3], bounds[2]]
    hexs_map.fit_bounds([sw, ne])

    branca_cmap = branca.colormap.LinearColormap(
        colors=[colors.to_hex(cmap(i)) for i in range(cmap.N)], 
        vmin=0, vmax=max_val
    )

    branca_cmap.caption = 'Total de atividades'
    branca_cmap.add_to(hexs_map)

    with od_map_output:
        od_map_output.clear_output(wait=True)
        display(hexs_map)

def on_value_change_od(change):
    if change['owner'] is granularidade_od:
        state['granularidade_od'] = change['new']
        update_map_od(state['granularidade_od'])
    elif change['owner'] is tipo:
        state['tipo'] = change['new']
        loc_id = None
        if granularidade_od.value == 'Zonas':
            loc_id = zone_dropdown_od.value
        elif granularidade_od.value == 'Distritos':
            loc_id = distritos_dropdown_od.value
        elif granularidade_od.value == 'Subprefeituras':
            loc_id = subprefeituras_dropdown_od.value
        update_hexs(loc_id, state['tipo'], state['periodo'])
    else:
        state['periodo'] = change['new']
        loc_id = None
        if granularidade_od.value == 'Zonas':
            loc_id = zone_dropdown_od.value
        elif granularidade_od.value == 'Distritos':
            loc_id = distritos_dropdown_od.value
        elif granularidade_od.value == 'Subprefeituras':
            loc_id = subprefeituras_dropdown_od.value
        update_hexs(loc_id, state['tipo'], state['periodo'])

granularidade_od.observe(on_value_change_od, names='value')
tipo.observe(on_value_change_od, names='value')
periodo.observe(on_value_change_od, names='value')

zone_dropdown_od.observe(lambda change: update_hexs(change['new'], state['tipo'], state['periodo']), names='value')
distritos_dropdown_od.observe(lambda change: update_hexs(change['new'], state['tipo'], state['periodo']), names='value')
subprefeituras_dropdown_od.observe(lambda change: update_hexs(change['new'], state['tipo'], state['periodo']), names='value')

update_map_od(state['granularidade_od'])

button_od.on_click(on_button_click_od)

display(label_od, HBox([label1_od, start_month_od, start_year_od]), HBox([label2_od, end_month_od, end_year_od]), button_od, notification_output_od, ui_output_od)

Label(value='Dados disponíveis: 01/2019 à 08/2024', layout=Layout(width='250px'))

Button(description='Selecionar período', style=ButtonStyle())

Output()

Output()

In [7]:
# # Criação de Outputs
# map_output_c = Output()
# dropdown_output_c = Output()
# notification_output_c = Output()
# ui_output_c = Output()

# months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# # Criação de Radio Buttons
# filtro_c = create_radio_buttons(['Nenhum', 'Idade', 'Gênero'], 'Filtrar por:')
# idade_c = create_radio_buttons(['Qualquer', '18-34', '35-54', '55-64', '65+'], 'Idade:')
# genero_c = create_radio_buttons(['Qualquer', 'Feminino', 'Masculino', 'Não especificado'], 'Gênero:')

# state = {'filtro_c': filtro_c.value, 'idade_c': idade_c.value, 'genero_c': genero_c.value}


# # Criação de Widgets para o filtro temporal
# layout_c = widgets.Layout(width='100px')
# start_month_c = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
# end_month_c = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
# start_year_c = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
# end_year_c = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
# label_c = create_labels('Dados disponíveis: 01/2019 à 08/2024', layout=widgets.Layout(width='250px'))
# label1_c = create_labels('Início do período', layout=widgets.Layout(width='100px'))
# label2_c = create_labels('Fim do período', layout=widgets.Layout(width='100px'))
# button_c = create_buttons('Selecionar período')


# def compute_trip_counts_c(grouped_df):
#     trips = grouped_df[['total_viagens', 'total_viagens_lazer', 'total_viagens_commute', 'total_pessoas',
#                         'total_pessoas_18_34', 'total_pessoas_35_54', 'total_pessoas_55_64', 'total_pessoas_o65']].sum().reset_index()
    
#     return trips

# # Obtenção de dados do período selecionado
# def get_data_c(df):
#     edges_in_districts = gpd.sjoin(edges, distritos, how="inner", predicate="intersects")
#     df_merged_districts = pd.merge(df, edges_in_districts[['edgeUID', 'ds_nome', 'ds_codigo']], left_on='edge_uid', right_on='edgeUID', how='inner')
#     grouped_edges_in_districts = df_merged_districts.groupby(['edgeUID', 'ds_codigo', 'ds_nome']).agg(
#         total_viagens=('total_viagens', 'sum'),
#         total_pessoas=('total_pessoas', 'sum'),
#         total_pessoas_18_34=('total_pessoas_18_34', 'sum'),
#         total_pessoas_35_54=('total_pessoas_35_54', 'sum'),
#         total_pessoas_55_64=('total_pessoas_55_64', 'sum'),
#         total_pessoas_o65=('total_pessoas_o65', 'sum'),
#     ).reset_index()
#     grouped_districts = df_merged_districts.groupby(['ds_nome'])
#     trips_per_district = compute_trip_counts_c(grouped_districts)

#     return trips_per_district

# # Mudança no mapa por granularidade
# def update_choropleth(filtro = None):
#     global combined_df_edges_c

#     if combined_df_edges_c.empty:
#         with notification_output_c:
#             notification_output_c.clear_output()
#             print('ERRO: nenhum período selecionado. Por favor selecione um período.')

#     choropleth_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
    
#     trips_per_district = get_data_c(combined_df_edges_c)

#     trips_per_district['log_total_viagens'] = np.cbrt(trips_per_district['total_viagens'].replace(0, 1))

#     distritos_merged = distritos.merge(trips_per_district, left_on="ds_nome", right_on="ds_nome")

#     choropleth = Choropleth(
#         geo_data=distritos_merged,
#         name='Choropleth - Total Viagens (Escala Logarítmica)',
#         data=trips_per_district,
#         columns=['ds_nome', 'log_total_viagens'],
#         key_on='feature.properties.ds_nome',
#         fill_color='YlOrBr',
#         fill_opacity=0.4,
#         line_opacity=0.2,
#         legend_name='Valor Representativo das Viagens por Distrito (Raiz Cúbica)'
#     ).add_to(choropleth_map)

#     GeoJson(
#         distritos_merged,
#         style_function=lambda feature: {
#             'weight': 0.5,
#             'color': 'black',
#             'fillColor': 'transparent',  
#             'fillOpacity': 0
#         },
#         tooltip=GeoJsonTooltip(
#             fields=['ds_nome', 'total_viagens'],
#             aliases=['Distrito:', 'Total de Viagens:'],
#             sticky=True,
#             localize=True,
#             labels=True
#         )
#     ).add_to(choropleth_map)

#     folium.LayerControl().add_to(choropleth_map)

#     with map_output_c:
#         map_output_c.clear_output(wait=True)
#         display(choropleth_map)


# #ações com mudança dos Radio Butttons
# def on_value_change_c(change):
#     if change['owner'] is filtro_c:
#         state['filtro_c'] = change['new']
#         if filtro_c.value == 'Nenhum':
#             idade_c.layout.display = 'none'
#             genero_c.layout.display = 'none'
#             update_choropleth()
#         elif filtro_c.value == 'Idade':
#             idade_c.layout.display = ''
#             genero_c.layout.display = 'none'
#             #update_choropleth(state['idade_c'])
#         elif filtro_c.value == 'Gênero':
#             idade_c.layout.display = 'none'
#             genero_c.layout.display = ''
#             #update_choropleth(state['genero_c'])
#     elif change['owner'] is idade_c:
#         state['idade_c'] = change['new']
#         update_choropleth(state['idade_c'])
#     elif change['owner'] is genero_c:
#         state['genero_c'] = change['new']
#         update_choropleth(state['genero_c'])
            
        
# # Radio Buttons Observers 
# filtro_c.observe(on_value_change_c, names='value')
# idade_c.observe(on_value_change_c, names='value')
# genero_c.observe(on_value_change_c, names='value')

# button_c.on_click(partial(on_button_click, notification=notification_output_c, ui=ui_output_c))

# idade_c.layout.display = 'none'
# genero_c.layout.display = 'none'

# display(label_c, HBox([label1_c, start_month_c, start_year_c]), HBox([label2_c, end_month_c, end_year_c]), button_c, notification_output_c, ui_output_c)
    
# cmap = colormaps.get_cmap('YlOrBr')

# #display(HBox([label1, start_month, start_year]), HBox([label2, end_month, end_year]), button, notification_output, HBox([granularidade, idade, genero]), dropdown_output, VBox([map_output, table_output, debug_output]))

<h2>Quartis</h2>

In [8]:

from folium import Html, Element

#quartis = create_radio_buttons(['Q1', 'Q2', 'Q3', 'Q4'], 'Quartil:')
quartile_map_output = Output()
notification_output_qt = Output()
ui_output_qt = Output()

# layout_qt = widgets.Layout(width='200px')
# quartis = create_dropdowns(['Q1', 'Q2', 'Q3', 'Q4'], '',Quartis', layout = layout_qt)
start_month_qt = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
end_month_qt = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
start_year_qt = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
end_year_qt = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
label_qt = create_labels('Dados disponíveis: 01/2019 à 08/2024', layout=widgets.Layout(width='250px'))
label1_qt = create_labels('Início do período', layout=widgets.Layout(width='100px'))
label2_qt = create_labels('Fim do período', layout=widgets.Layout(width='100px'))
button_qt = create_buttons('Selecionar período')

def on_button_click_qt(b):
    stt_y = start_year_qt.value 
    end_y = end_year_qt.value 
    stt_m = start_month_qt.value
    end_m = end_month_qt.value

    with notification_output_qt:
        notification_output_qt.clear_output(wait=True)
        
    with ui_output_qt:
        ui_output_qt.clear_output(wait=True)

    start_date = f"{stt_y} {stt_m}"
    end_date = f"{end_y} {end_m}"

    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)

    limit_2024 = pd.to_datetime('2024-08')
    if end_date_dt > limit_2024:
        with notification_output_qt:
            print("Erro: Os dados estão disponíveis apenas até 08/2024.")
        return

    if end_date_dt < start_date_dt:
        with notification_output_qt:
            print("Erro: Período inválido. Verifique início e fim do período.")
        return

    matching_files = []
    
    for year in range(int(stt_y), int(end_y) + 1):
        for month in months:
            filename = f"{year} {month}.csv"
            file_path = os.path.join('csvs-editados', filename)
            if year > int(stt_y) or (year == int(stt_y) and month >= stt_m):
                if year < int(end_y) or (year == int(end_y) and month <= end_m):
                    if os.path.isfile(file_path):
                        matching_files.append(file_path)

    if matching_files:
        try:
            with notification_output_qt:
                print("Obtendo dados...")
            
            df_list = [pd.read_csv(file) for file in matching_files] 
            concatenated_df = pd.concat(df_list, ignore_index=True)

            global combined_df_edges_quartile
            combined_df_edges_quartile = concatenated_df 
            
            with notification_output_qt:
                print(f"Dados de {stt_m}/{stt_y} a {end_m}/{end_y} obtidos.")

            # with quartile_map_output:
            #     quartile_map_output.clear_output(wait=True)

            # with ui_output_qt:
            # #     #display(update_map_button, quartile_map_output)
            #      display(quartile_map_output)
            update_quartile()

            
        except Exception as e:
            with notification_output_qt:
                print(f"Erro na obtenção de dados: {e}")
       
    else:
        with notification_output_qt:
            print("Nenhum arquivo encontrado.")


def get_quartile_data(df):
    trips_per_edge = df.groupby('edge_uid').agg({'total_viagens': 'sum'}).reset_index()
    trips_per_edge['quartile'] = pd.qcut(trips_per_edge['total_viagens'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    edges_in_quartiles = pd.merge(edges, trips_per_edge[['edge_uid', 'quartile']], left_on='edgeUID', right_on='edge_uid', how='inner')

    return edges_in_quartiles

# quartile_colors = {
#     'Q1': 'green',
#     'Q2': 'blue',
#     'Q3': 'orange',
#     'Q4': 'red'
# }

def update_quartile():
    # with quartile_map_output:
    #     quartile_map_output.clear_output(wait=True)

    global combined_df_edges_quartile

    # if combined_df_edges_quartile.empty:
    #     with notification_output_qt:
    #         print("Dados não carregados. Por favor, selecione um período e tente novamente.")
    #     return

    data = combined_df_edges_quartile

    quartiles_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
    
    edges_in_quartiles = get_quartile_data(data)
    q1 = edges_in_quartiles[edges_in_quartiles['quartile'] == 'Q1']
    q2 = edges_in_quartiles[edges_in_quartiles['quartile'] == 'Q2']
    q3 = edges_in_quartiles[edges_in_quartiles['quartile'] == 'Q3']
    q4 = edges_in_quartiles[edges_in_quartiles['quartile'] == 'Q4']

    gdf_q1 = gpd.GeoDataFrame(q1, geometry='geometry')
    gdf_q2 = gpd.GeoDataFrame(q2, geometry='geometry')
    gdf_q3 = gpd.GeoDataFrame(q3, geometry='geometry')
    gdf_q4 = gpd.GeoDataFrame(q4, geometry='geometry')

    folium.GeoJson(
        gdf_q1,
        style_function=lambda feature: {
                'weight': 2,
                'color': '#ADD8E6'
            },
        name="Q1",
        show=False 
    ).add_to(quartiles_map)
    
    folium.GeoJson(
        gdf_q2,
        style_function=lambda feature: {
                'weight': 2,
                'color': '#87CEEB'
            },
        name="Q2",
        show=False 
    ).add_to(quartiles_map)
    
    folium.GeoJson(
        gdf_q3,
        style_function=lambda feature: {
                'weight': 2,
                'color': '#4682B4'
            },
        name="Q3",
        show=False 
    ).add_to(quartiles_map)
    
    folium.GeoJson(
        gdf_q4,
        style_function=lambda feature: {
                'weight': 2,
                'color': '#00008B'
            },
        name="Q4",
        show=False 
    ).add_to(quartiles_map)
    

    # with notification_output_qt:
    #     print(filtered_edges)

    folium.GeoJson(
        sp,
        style_function=lambda feature: {
            'weight': 1,
            'color': 'black',
            'fillOpacity': 0,
        },
        name='São Paulo Shape'
    ).add_to(quartiles_map)

    legend_html = """
    <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: auto; 
                 background-color: white; border:2px solid grey; z-index:9999; 
                 font-size: 14px; padding: 10px;">
        <h4>Quartis</h4>
        <div style="display: flex; align-items: center; margin-bottom: 5px;">
            <div style="width: 30px; height: 30px; background-color: #ADD8E6; margin-right: 10px;"></div>
            <span>Q1</span>
        </div>
        <div style="display: flex; align-items: center; margin-bottom: 5px;">
            <div style="width: 30px; height: 30px; background-color: #87CEEB; margin-right: 10px;"></div>
            <span>Q2</span>
        </div>
        <div style="display: flex; align-items: center; margin-bottom: 5px;">
            <div style="width: 30px; height: 30px; background-color: #4682B4; margin-right: 10px;"></div>
            <span>Q3</span>
        </div>
        <div style="display: flex; align-items: center;">
            <div style="width: 30px; height: 30px; background-color: #00008B; margin-right: 10px;"></div>
            <span>Q4</span>
        </div>
    </div>
    """
    
    # Add the legend to the map
    quartiles_map.get_root().html.add_child(Element(legend_html))

    folium.LayerControl().add_to(quartiles_map)
    
    with quartile_map_output:
        # quartile_map_output.clear_output(wait=True)
        # time.sleep(0.1)
        output_directory = 'Map Outputs'
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        quartiles_map.save(os.path.join(output_directory, f"{start_month_qt.value}-{start_year_qt.value} a {end_month_qt.value}-{end_year_qt.value}.html"))
        #display(quartiles_map)

    with notification_output_qt:
        print(f"Mapa '{start_month_qt.value}-{start_year_qt.value} a {end_month_qt.value}-{end_year_qt.value}'.html salvo em 'Map Outputs'")
    
# quartis.observe(update_quartile, names='value')

#update_quartile()

#update_map_button = widgets.Button(description="Atualizar Mapa")
#update_map_button.on_click(lambda x: update_quartile())

button_qt.on_click(on_button_click_qt)
display(label_qt, HBox([label1_qt, start_month_qt, start_year_qt]), HBox([label2_qt, end_month_qt, end_year_qt]), button_qt, notification_output_qt, ui_output_qt, quartile_map_output)

Label(value='Dados disponíveis: 01/2019 à 08/2024', layout=Layout(width='250px'))

Button(description='Selecionar período', style=ButtonStyle())

Output()

Output()

Output()

<h2>Velocidades Médias</h2>

In [9]:
# #speeds = create_radio_buttons(['< 10', '10-15', '15-20', '20-25', '25-30', '> 30'], 'Faixas de velocidade em km/h:')
# average_speed_map_output = Output()
# debug3 = Output()
# notification_output_vel = Output()
# ui_output_vel = Output()

# layout_vel = widgets.Layout(width='300px')
# speeds = create_dropdowns(['< 10', '10-15', '15-20', '20-25', '25-30', '> 30'], 'Faixas de velocidade em km/h:', layout = layout_vel)
# start_month_vel = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
# end_month_vel = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
# start_year_vel = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
# end_year_vel = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
# label_vel = create_labels('Dados disponíveis: 01/2019 à 08/2024', layout=widgets.Layout(width='250px'))
# label1_vel = create_labels('Início do período', layout=widgets.Layout(width='100px'))
# label2_vel = create_labels('Fim do período', layout=widgets.Layout(width='100px'))
# button_vel = create_buttons('Selecionar período')
# #display(speeds, average_speed_map_output, debug3)

# def on_button_click_vel(b):
    
#     stt_y = start_year_vel.value 
#     end_y = end_year_vel.value 
#     stt_m = start_month_vel.value
#     end_m = end_month_vel.value
    
#     with notification_output_vel:
#         notification_output_vel.clear_output()

#     with ui_output_vel:
#         ui_output_vel.clear_output()

#     start_date = f"{stt_y} {stt_m}"
#     end_date = f"{end_y} {end_m}"

#     start_date_dt = pd.to_datetime(start_date)
#     end_date_dt = pd.to_datetime(end_date)

#     limit_2024 = pd.to_datetime('2024-08')
#     if end_date_dt > limit_2024:
#         with notification_output_vel:
#             print("Erro: Os dados estão disponíveis apenas até 08/2024.")
#         return

#     if end_date_dt < start_date_dt:
#         with notification_output_vel:
#             print("Erro: Período inválido. Verifique início e fim do período.")
#         return

#     matching_files = []
    
#     for year in range(int(stt_y), int(end_y) + 1):
#         for month in months:
#             filename = f"{year} {month}.csv"
#             file_path = os.path.join('csvs-editados', filename)
#             if year > int(stt_y) or (year == int(stt_y) and month >= stt_m):
#                 if year < int(end_y) or (year == int(end_y) and month <= end_m):
#                     if os.path.isfile(file_path):
#                         matching_files.append(file_path)

#     if matching_files:
#         try:
#             with notification_output_vel:
#                 print("Obtendo dados...")
            
#             df_list = [pd.read_csv(file) for file in matching_files] 
#             concatenated_df = pd.concat(df_list, ignore_index=True)

#             global combined_df_edges_vel
#             combined_df_edges_vel = concatenated_df 
            
#             #output_file = "concatenated_data.csv" 
#             #concatenated_df.to_csv(output_file, index=False) 
#             with notification_output_vel:
#                 print(f"Dados de {stt_m}/{stt_y} a {end_m}/{end_y} obtidos.")
            
#             with ui_output_vel:
#                  display(speeds, average_speed_map_output)
            
#         except Exception as e:
#             with notification_output_vel:
#                 print(f"Erro na obtenção de dados: {e}")
       
#     else:
#         with notification_output_vel:
#             print("Nenhum arquivo encontrado.")

# def update_speed(change):
#     average_speed_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
#     speed_range = change['new']

#     global combined_df_edges_vel

#     if speed_range == '< 10':
#         sub_df = combined_df_edges_vel[combined_df_edges_vel['velocidade_media'] < 10]
#     elif speed_range == '10-15':
#         sub_df = combined_df_edges_vel[(combined_df_edges_vel['velocidade_media'] >= 10) & (combined_df_edges_vel['velocidade_media'] < 15)]
#     elif speed_range == '15-20':
#         sub_df = combined_df_edges_vel[(combined_df_edges_vel['velocidade_media'] >= 15) & (combined_df_edges_vel['velocidade_media'] < 20)]
#     elif speed_range == '20-25':
#         sub_df = combined_df_edges_vel[(combined_df_edges_vel['velocidade_media'] >= 20) & (combined_df_edges_vel['velocidade_media'] < 25)]
#     elif speed_range == '25-30':
#         sub_df = combined_df_edges_vel[(combined_df_edges_vel['velocidade_media'] >= 25) & (combined_df_edges_vel['velocidade_media'] < 30)]
#     elif speed_range == '> 30':
#         sub_df = combined_df_edges_vel[combined_df_edges_vel['velocidade_media'] > 30]

#     edges_by_speed = edges[edges['edgeUID'].isin(sub_df['edge_uid'])]

#     folium.GeoJson(
#         sp,
#         style_function=lambda feature: {
#             'weight': 1,
#             'color': 'black',
#             'fillOpacity': 0,
#         }
#     ).add_to(average_speed_map)

#     folium.GeoJson(
#         edges_by_speed,
#         style_function=lambda feature: {
#             'weight': 1,
#             'color': 'brown',
#         }
#     ).add_to(average_speed_map)

#     with average_speed_map_output:
#         average_speed_map_output.clear_output(wait=True)
#         display(average_speed_map)

# speeds.observe(update_speed, names='value')
# button_vel.on_click(on_button_click_vel)
# display(label_vel, HBox([label1_vel, start_month_vel, start_year_vel]), HBox([label2_vel, end_month_vel, end_year_vel]), button_vel, notification_output_vel, ui_output_vel)

In [10]:
# Criação de Outputs
map_output_g = Output()
notification_output_g = Output()
ui_output_g = Output()

months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Criação de Widgets para o filtro temporal
layout = widgets.Layout(width='100px')
start_month_g = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
end_month_g = create_dropdowns(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '', layout = layout)
start_year_g = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
end_year_g = create_dropdowns(['2019', '2020', '2021', '2022', '2023', '2024'], '', layout = layout)
label_g = create_labels('Dados disponíveis: 01/2019 à 08/2024', layout=widgets.Layout(width='250px'))
label1_g = create_labels('Início do período', layout=widgets.Layout(width='100px'))
label2_g = create_labels('Fim do período', layout=widgets.Layout(width='100px'))
button_g = create_buttons('Selecionar período')

#Filtra o período ao pressionar o botão
def on_button_click_g(b):
    stt_y = start_year_g.value 
    end_y = end_year_g.value 
    stt_m = start_month_g.value
    end_m = end_month_g.value

    with notification_output_g:
        notification_output_g.clear_output(wait=True)
        
    with ui_output_g:
        ui_output_g.clear_output(wait=True)

    start_date = f"{stt_y} {stt_m}"
    end_date = f"{end_y} {end_m}"

    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)

    limit_2024 = pd.to_datetime('2024-08')
    if end_date_dt > limit_2024:
        with notification_output_g:
            print("Erro: Os dados estão disponíveis apenas até 08/2024.")
        return

    if end_date_dt < start_date_dt:
        with notification_output_g:
            print("Erro: Período inválido. Verifique início e fim do período.")
        return

    matching_files = []
    
    for year in range(int(stt_y), int(end_y) + 1):
        for month in months:
            filename = f"{year} {month}.csv"
            file_path = os.path.join('csvs-editados', filename)
            if year > int(stt_y) or (year == int(stt_y) and month >= stt_m):
                if year < int(end_y) or (year == int(end_y) and month <= end_m):
                    if os.path.isfile(file_path):
                        matching_files.append(file_path)

    if matching_files:
        try:
            with notification_output_g:
                print("Obtendo dados...")
            
            df_list = [pd.read_csv(file) for file in matching_files] 
            concatenated_df = pd.concat(df_list, ignore_index=True)

            global combined_df_edges_g
            combined_df_edges_g = concatenated_df 
            
            with notification_output_g:
                print(f"Dados de {stt_m}/{stt_y} a {end_m}/{end_y} obtidos.")

            with ui_output_g:
                display(map_output_g)
            
        except Exception as e:
            with notification_output_g:
                print(f"Erro na obtenção de dados: {e}")
       
    else:
        with notification_output_g:
            print("Nenhum arquivo encontrado.")

    update_total()


# Mudança de zona/distrito/subprefeitura a partir da seleção do usuário
def update_total():
    global combined_df_edges_g
    if combined_df_edges_g.empty:
        with notification_output_g:
            notification_output_g.clear_output()
            print('ERRO: nenhum período selecionado. Por favor selecione um período.')
    
    total_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)

    #grouped_edges, trips_per_loc = get_data(combined_df_edges, "Zonas")

    df_g = combined_df_edges_g

    with notification_output_g:
        print(len(df_g))

    grouped = df_g.groupby(['edge_uid']).agg(
            total_viagens=('total_viagens', 'sum'),
        ).reset_index()

    with notification_output_g:
        print(len(grouped))

    #selected_edges = grouped_edges[grouped_edges['edgeUID'].isin(filtered_edges['edgeUID'])]
    with notification_output_g:
        print('here')
    grouped['log_total_viagens'] = np.cbrt(grouped['total_viagens'].replace(0, 1))    
    merged_edges = edges.merge(grouped, left_on='edgeUID', right_on='edge_uid') #inverte ordem
    #merged_edges['log_total_viagens'] = np.cbrt(merged_edges['total_viagens'].replace(0, 1))

    with notification_output_g:
        print(len(merged_edges))

    min_val, max_val = merged_edges['log_total_viagens'].min(), merged_edges['log_total_viagens'].max()
    
    # merged_edges['color'] = merged_edges.apply(
    #     lambda row: get_color(merged_edges, row['edgeUID'], min_val, max_val, 'edgeUID', 'log_total_viagens'),
    # axis=1
    # )

    with notification_output_g:
        print('here3')

    with notification_output_g:
        print(merged_edges)
        print(grouped)

    folium.GeoJson(
            merged_edges,
            style_function=lambda feature: {
                'weight': 2,
                'color': get_color(grouped, feature['properties']['edge_uid'], min_val, max_val, 'edge_uid', 'log_total_viagens')
            }
            # tooltip=folium.GeoJsonTooltip(
            #     fields=['edge_uid', 'total_viagens'],
            #     aliases=['ID do Edge:', 'Total de viagens:'],
            #     sticky=True,
            #     localize=True,
            #     labels=True,
            #     formatters={
            #         'edge_uid': 'function() { return this.edgeUID; }',
            #     }
            # ),
            # name='Arestas de São Paulo'
        ).add_to(total_map)

    with notification_output_g:
        print('here4')

    # GeoJson(
    #     merged_edges,
    #     style_function=lambda feature: {
    #         'weight': 0.5,
    #         'color': 'black',
    #         'fillColor': 'transparent',  
    #         'fillOpacity': 0
    #     }
    #     ,
    #     tooltip=GeoJsonTooltip(
    #         fields=['edge_uid', 'log_total_viagens'],
    #         aliases=['ID Aresta:', 'Total de Viagens:'],
    #         sticky=True,
    #         localize=True,
    #         labels=True
    #     )
    # ).add_to(total_map)


    # Create the GeoJson layer using the precomputed color
    # folium.GeoJson(
    #     merged_edges,
    #     style_function=lambda feature: {
    #         'weight': 2,
    #         'color': feature['properties']['color']
    #     },
    #     tooltip=folium.GeoJsonTooltip(
    #             fields=['edgeUID', 'total_pessoas'],
    #             aliases=['ID do Edge:', 'Total de viagens:'],
    #             sticky=True,
    #             localize=True,
    #             labels=True,
    #             formatters={
    #                 'edgeUID': 'function() { return this.edgeUID; }',
    #             }
    #         ),
    #     name='Arestas de São Paulo'
    # ).add_to(total_map)

    

    c_ciclovias = ciclovias[ciclovias['programa'].str.contains("CICLOVIA", case=False, na=False)]
    c_ciclofaixas = ciclovias[ciclovias['programa'].str.contains("CICLOFAIXA", case=False, na=False)]

    folium.GeoJson(
        c_ciclovias,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'green'
            },
        name="Ciclovias",
        show=False 
    ).add_to(total_map)

    folium.GeoJson(
        c_ciclofaixas,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'blue'
            },
        name="Ciclofaixas",
        show=False 
    ).add_to(total_map)

    folium.GeoJson(
        ciclorrotas,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'purple'
            },
        name="Ciclorrotas",
        show=False 
    ).add_to(total_map)

    folium.GeoJson(
        conexoes,
        style_function=lambda feature: {
                'weight': 2,
                'color': 'pink'
            },
        name="Conexões",
        show=False 
    ).add_to(total_map)
    
    folium.LayerControl().add_to(total_map)

    with notification_output_g:
        print('here5')

    branca_cmap = branca.colormap.LinearColormap(
        colors=[colors.to_hex(cmap(i)) for i in range(cmap.N)], 
        vmin=0, vmax=max_val
    )

    branca_cmap.caption = 'Total de viagens'
    branca_cmap.add_to(total_map)

    with map_output_g:
        output_directory = 'Map Outputs'
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        total_map.save(os.path.join(output_directory, f"Totalsp - {start_month_g.value}-{start_year_g.value} a {end_month_g.value}-{end_year_g.value}.html"))
        with notification_output_g:
            print('here6')

        # map_output_g.clear_output(wait=True)
        # display(total_map)

button_g.on_click(on_button_click_g)
    
#cmap = colormaps.get_cmap('YlOrBr')

#display(HBox([label1, start_month, start_year]), HBox([label2, end_month, end_year]), button, notification_output, HBox([granularidade, idade, genero]), dropdown_output, VBox([map_output, table_output, debug_output]))
display(label_g, HBox([label1_g, start_month_g, start_year_g]), HBox([label2_g, end_month_g, end_year_g]), button_g, notification_output_g, ui_output_g)

Label(value='Dados disponíveis: 01/2019 à 08/2024', layout=Layout(width='250px'))

Button(description='Selecionar período', style=ButtonStyle())

Output()

Output()